<a href="https://colab.research.google.com/github/lordsnat/bootcamp_ML-Projeto-de-M-tricas-de-Avalia-o/blob/main/metricas_avaliacao_Dio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline

import os

#if using Theano with GPU
os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [3]:
#Dataset composto por uma pasta zipada dentro do drive

from pathlib import Path

!unzip -q "/content/drive/MyDrive/Dio.me/Transfer_Learning/PetImages.zip" -d /content

In [4]:
#Código verifica se há erros de formato nas imagens extraídas do dataset

import tensorflow as tf
import pathlib, os

root = pathlib.Path("/content/PetImages")  # ajuste se for outra pasta
valid_ext = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}

bad = []
total_ok = 0

for p in root.rglob("*"):
    if not p.is_file():
        continue
    ext = p.suffix.lower()

    #Remove arquivos com extensões não suportadas, ocultos, tamanho 0, etc.
    if (ext not in valid_ext) or p.name.startswith(("._", ".")) or os.path.getsize(p) == 0:
        bad.append((str(p), "unsupported/hidden/empty"))
        continue

    try:
        raw = tf.io.read_file(str(p))
        #Tenta decodificar explicitamente conforme a extensão (mais estável)
        if ext in {".jpg", ".jpeg"}:
            _ = tf.image.decode_jpeg(raw, channels=3)
        elif ext == ".png":
            _ = tf.image.decode_png(raw, channels=3)
        elif ext == ".gif":
            _ = tf.image.decode_gif(raw)  #Retorna [num_frames, H, W, 3]
        elif ext == ".bmp":
            _ = tf.image.decode_bmp(raw)
        total_ok += 1
    except Exception as e:
        bad.append((str(p), repr(e)))

print(f"OK (decodificou): {total_ok}")
print(f"Arquivos problemáticos: {len(bad)}")
for path, err in bad[:25]:
    print("BAD:", path, "->", err)

OK (decodificou): 24824
Arquivos problemáticos: 178
BAD: /content/PetImages/Dog/7739.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/8641.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/719.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/11692.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/4203.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/3927.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/9640.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/4924.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/6503.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/6855.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/11166.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/6500.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/7128.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/7718.jpg -> InvalidArgumentError()
BAD: /content/PetImages/Dog/7652.jpg -> InvalidArgumentError()
BA

In [5]:
#Remove os casos problemáticos
for path, _ in bad:
    try:
        os.remove(path)
    except Exception as e:
        print("Falha ao remover:", path, e)
print("Remoção concluída.")

Remoção concluída.


In [6]:
#Como a base PetImages é muito pesada, cria um subset menor com algumas amostras desse dataset
import shutil, random

orig = pathlib.Path("/content/PetImages")
small = pathlib.Path("/content/PetImages_small")
for cls in ["Cat", "Dog"]:
    (small/cls).mkdir(parents=True, exist_ok=True)

#Pegue no máx N por classe
N = 1500
random.seed(42)

for cls in ["Cat", "Dog"]:
    files = [p for p in (orig/cls).glob("*") if p.is_file()]
    random.shuffle(files)
    kept = 0
    for p in files:
        if kept >= N: break
        #Copia apenas extensões suportadas
        if p.suffix.lower() in {".jpg", ".jpeg", ".png", ".bmp", ".gif"} and os.path.getsize(p) > 0:
            shutil.copy2(p, small/cls/p.name)
            kept += 1

print("Subset criado em:", small)

Subset criado em: /content/PetImages_small


In [7]:
root_dir = "/content/PetImages_small"

# 1) Cria os datasets brutos e capture class_names
train_raw = tf.keras.utils.image_dataset_from_directory(
    root_dir, validation_split=0.15, subset="training",
    seed=42, image_size=(224,224), batch_size=8, shuffle=True
)
val_raw = tf.keras.utils.image_dataset_from_directory(
    root_dir, validation_split=0.15, subset="validation",
    seed=42, image_size=(224,224), batch_size=8
)

class_names = train_raw.class_names   # <-- capture aqui
print("Classes:", class_names)

# 2) Aplica cache/prefetch/ignore_errors
train_ds = train_raw.cache().prefetch(tf.data.AUTOTUNE).apply(tf.data.experimental.ignore_errors())
val_ds   = val_raw.cache().prefetch(tf.data.AUTOTUNE).apply(tf.data.experimental.ignore_errors())

Found 3000 files belonging to 2 classes.
Using 2550 files for training.
Found 3000 files belonging to 2 classes.
Using 450 files for validation.


Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.


Classes: ['Cat', 'Dog']


*Camadas de preparação + carregar VGG16*

In [8]:
num_classes = len(class_names)
print("Número de classes:", num_classes)

Número de classes: 2


In [9]:
from tensorflow.keras import layers, models, applications

IMG_SIZE = (224, 224)
NUM_CLASSES = num_classes

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

preprocess = layers.Rescaling(1./255)

#Base VGG16 pré-treinada no ImageNet, sem o topo
base = applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=IMG_SIZE + (3,)
)
base.trainable = False  #Fase 1: congelado

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


*Montar o modelo (cabeça de classificação)*

In [10]:
inputs = layers.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = preprocess(x)
x = base(x, training=False)                #Mantém training=False com base congelada
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,714 (56.14 MB)

 Trainable params: 1,026 (4.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [11]:
#Callbacks úteis para treinamento do modelo

ckpt_cb = tf.keras.callbacks.ModelCheckpoint(
    "vgg16_feat_extract.keras", monitor="val_accuracy",
    save_best_only=True, mode="max"
)
es_cb = tf.keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True, monitor="val_accuracy", mode="max"
)
rlr_cb = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.5, patience=2, monitor="val_loss", mode="min"
)

In [12]:
#Treinar modelo
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[ckpt_cb, es_cb, rlr_cb]
)

Epoch 1/10
    319/Unknown 23s 52ms/step - accuracy: 0.6148 - loss: 0.6511

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


319/319 ━━━━━━━━━━━━━━━━━━━━ 27s 65ms/step - accuracy: 0.6150 - loss: 0.6509 - val_accuracy: 0.8089 - val_loss: 0.4884 - learning_rate: 0.0010
Epoch 2/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 34s 58ms/step - accuracy: 0.7941 - loss: 0.5009 - val_accuracy: 0.8511 - val_loss: 0.4083 - learning_rate: 0.0010
Epoch 3/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 21s 64ms/step - accuracy: 0.8240 - loss: 0.4421 - val_accuracy: 0.8578 - val_loss: 0.3723 - learning_rate: 0.0010
Epoch 4/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 39s 59ms/step - accuracy: 0.8458 - loss: 0.4010 - val_accuracy: 0.8689 - val_loss: 0.3465 - learning_rate: 0.0010
Epoch 5/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 21s 60ms/step - accuracy: 0.8488 - loss: 0.3769 - val_accuracy: 0.8756 - val_loss: 0.3273 - learning_rate: 0.0010
Epoch 6/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 20s 59ms/step - accuracy: 0.8480 - loss: 0.3662 - val_accuracy: 0.8778 - val_loss: 0.3154 - learning_rate: 0.0010
Epoch 7/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - accuracy: 0.8569 - loss: 0.3519

*Métricas de performance*

In [13]:
test_loss, test_acc = model.evaluate(val_ds)
print("Test accuracy:", test_acc)
print("Test lost:", test_loss)

57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.8905 - loss: 0.2879
Test accuracy: 0.8866666555404663
Test lost: 0.2805069386959076


In [14]:
#Função para obtenção da imagem
def get_image(path):
    #Carrega e redimensiona
    img = keras.preprocessing.image.load_img(path, target_size=(224, 224))
    #Para array (H, W, 3)
    x = keras.utils.img_to_array(img)

    #Adiciona dimensão de batch: (1, 224, 224, 3)
    x = np.expand_dims(x, axis=0)
    return img, x

In [15]:
#Teste manual de performance: imagem gato
img, x = get_image('/content/drive/MyDrive/Dio.me/Transfer_Learning/teste_gato_meng.jpeg')

probs = model.predict(x)
print(class_names)
print("Probabilidades:", probs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 693ms/step
['Cat', 'Dog']
Probabilidades: [[0.64447314 0.35552686]]


In [16]:
# Teste manual de performance: imagem cachorro
img, x = get_image('/content/drive/MyDrive/Dio.me/Transfer_Learning/teste_cachorro_clara.jpeg')

probs = model.predict(x)
print(class_names)
print("Probabilidades:", probs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
['Cat', 'Dog']
Probabilidades: [[0.12839171 0.87160826]]


*Preparando base de validação*

In [17]:
val_eval = val_ds.map(lambda x,y: (tf.cast(x, tf.float32), y))
val_eval = val_eval.map(lambda x,y: (tf.keras.applications.vgg16.preprocess_input(x), y))

# Pega só os rótulos por batch e concatena (cada y tem shape (batch,))
y_true = np.concatenate([
    y for y in val_eval.map(lambda x,y: y).as_numpy_iterator()
])

*Calculando as métricas*

In [18]:
# predições
y_prob = model.predict(val_eval, verbose=0)
y_pred = y_prob.argmax(axis=1)

from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))
print(cm)

[[190  31]
 [ 23 206]]


Temos que 0 representa a classe "Cat" e 1 representa a classe "Dog".
No cálculo das métricas de recall, acurácia e especificiada, consideraremos que 1 (cachorro) é o valor positivo e 0 (gato) é o valor negativo.

Logo:
* VP => a imagem é de cachorro e o modelo previu um cachorro
* VN => a imagem é de um gato e o modelo previu um gato
* FP => a imagem é de um gato e o modelo previu um cachorro
* FN => a imagem é de um cachorro e o modelo previu um gato

In [19]:
#Recall
#VP/(VP+FN)
recall = cm[1,1]/(cm[1,1]+cm[1,0])
recall

np.float64(0.8995633187772926)

In [20]:
#Acurácia
#(VP+VN)/(VP + VN + FP + FP)
acuracia = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1] + cm[0,1] + cm[1,0])
acuracia

np.float64(0.88)

In [21]:
#Precisão
#VP/(VP+FP)
precisao = cm[1,1]/(cm[1,1]+cm[0,1])
precisao

np.float64(0.869198312236287)

In [22]:
#Especificidade
#VN/(VN+FP)
especificidade = cm[0,0]/(cm[0,0]+cm[0,1])
especificidade

np.float64(0.8597285067873304)

In [23]:
#F-score
#2(PxS)/(P+S)
p = precisao
s = especificidade
f_score = 2*(p*s)/(p+s)
f_score

np.float64(0.8644374751534962)

*Fine-tuning (descongelar topo da VGG16)
Descongele algumas camadas superiores para ajustar a base ao seu dataset.*

In [24]:
# Descongela a partir de um bloco (ex.: último bloco conv da VGG16)
for layer in base.layers:
    layer.trainable = False
for layer in base.layers[-4*3:]:  # Aprox. últimas 4 camadas conv
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # LR menor no fine-tuning
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[ckpt_cb, es_cb, rlr_cb]
)

Epoch 1/10
    319/Unknown 43s 118ms/step - accuracy: 0.8942 - loss: 0.2421

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


319/319 ━━━━━━━━━━━━━━━━━━━━ 47s 130ms/step - accuracy: 0.8943 - loss: 0.2419 - val_accuracy: 0.9289 - val_loss: 0.1403 - learning_rate: 1.0000e-05
Epoch 2/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 40s 124ms/step - accuracy: 0.9615 - loss: 0.0987 - val_accuracy: 0.9511 - val_loss: 0.1349 - learning_rate: 1.0000e-05
Epoch 3/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 40s 126ms/step - accuracy: 0.9709 - loss: 0.0727 - val_accuracy: 0.9600 - val_loss: 0.0971 - learning_rate: 1.0000e-05
Epoch 4/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 40s 125ms/step - accuracy: 0.9805 - loss: 0.0527 - val_accuracy: 0.9667 - val_loss: 0.0990 - learning_rate: 1.0000e-05
Epoch 5/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 39s 123ms/step - accuracy: 0.9813 - loss: 0.0628 - val_accuracy: 0.9556 - val_loss: 0.1161 - learning_rate: 1.0000e-05
Epoch 6/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 40s 125ms/step - accuracy: 0.9890 - loss: 0.0269 - val_accuracy: 0.9711 - val_loss: 0.0648 - learning_rate: 5.0000e-06
Epoch 7/10
319/319 ━━━━━━━━━━━━━━━━━━━━ 40s 123ms/step - 

*Testes de uso pós fine tunning*

In [25]:
# Exemplo de uso cachorro
img, x = get_image('/content/drive/MyDrive/Dio.me/Transfer_Learning/teste_cachorro_clara.jpeg')
probabilities = model.predict(x)
print(class_names)
probabilities

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
['Cat', 'Dog']


array([[3.0400799e-10, 1.0000000e+00]], dtype=float32)

In [26]:
# Exemplo de uso gato
img, x = get_image('/content/drive/MyDrive/Dio.me/Transfer_Learning/teste_gato_gugu.jpeg')
probabilities = model.predict(x)

print(class_names)
probabilities

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
['Cat', 'Dog']


array([[9.999169e-01, 8.312655e-05]], dtype=float32)

*Calculando novamente as métricas pós fine tuning*

In [27]:
# predições
y_prob = model.predict(val_eval, verbose=0)
y_pred = y_prob.argmax(axis=1)

from sklearn.metrics import confusion_matrix, classification_report
cm_pt = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))
print(cm_pt)

[[208  13]
 [  2 227]]


In [28]:
#Recall
#VP/(VP+FN)
recall_pt = cm_pt[1,1]/(cm_pt[1,1]+cm_pt[1,0])
recall_pt

np.float64(0.9912663755458515)

In [29]:
recall

np.float64(0.8995633187772926)

In [30]:
#Acurácia
#(VP+VN)/(VP + VN + FP + FP)
acuracia_pt = (cm_pt[0,0] + cm_pt[1,1])/(cm_pt[0,0] + cm_pt[1,1] + cm_pt[0,1] + cm_pt[1,0])
acuracia_pt

np.float64(0.9666666666666667)

In [31]:
acuracia

np.float64(0.88)

In [32]:
#Precisão
#VP/(VP+FP)
precisao_pt = cm_pt[1,1]/(cm_pt[1,1]+cm_pt[0,1])
precisao_pt

np.float64(0.9458333333333333)

In [33]:
precisao

np.float64(0.869198312236287)

In [34]:
#Especificidade
#VN/(VN+FP)
especificidade_pt = cm_pt[0,0]/(cm_pt[0,0]+cm_pt[0,1])
especificidade_pt

np.float64(0.9411764705882353)

In [35]:
especificidade

np.float64(0.8597285067873304)